In [1]:
# Global inversion with empymod

#%matplotlib notebook

In [1]:
import empymod
import numpy as np
#import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed

In [15]:

# Receivers geometry

offsets = np.array([2, 4, 8]) # in meters
dip = np.array([0, 90])

Hreceivers = [offsets, offsets*0, 0, 0, 0]
Vreceivers = [offsets, offsets*0, 0, 0, 90]

# Source geometry

Hsource = [0, 0, 0 ,0 , 0]
Vsource = [0, 0, 0, 0, 90]

# Frequency

freq = 9000

In [16]:
# True data

res = [2e14, 10, 20, 10]
depth = [0, 2, 5]

HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
PRP = empymod.loop(Hsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop')

Zdata = np.hstack((HCP, VCP, PRP))



:: empymod END; runtime = 0:00:00.007634 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.004076 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.002617 :: 1 kernel call(s)



In [24]:
# sampling of depth and conductivities
nsl = 5

s0 = -2 # minimum conductivity in S/m
s1 = -0.8 # maximum conductivity in S/m
# conductivities array
conds = np.logspace(s0, s1, nsl)

th0 = 0.1 # minimum thickness in m
th1 = 5   # maximum thickness in m
# thickness array
thicks = np.linspace(th0, th1, nsl)

# Array to store values

#Zcube = np.zeros((nsl, nsl, nsl, nsl, nsl, 9), dtype = 'complex') # 9 coil geometries, 5 parameters 

In [25]:
# Loop to create hypercube

def forward_parallel(is1, is2, is3, it1, it2):
    time.sleep(0.1)
    
    res[1] = 1/conds[is1] # set resistivity of first layer
    res[2] = 1/conds[is2] # set resistivity of second layer
    res[3] = 1/conds[is3] # set resistivity of third layer
    depth[1] = thicks[it1] # set thickness of first layer
    depth[2] = depth[1] + thicks[it2] # set thickness of second layer

    # Compute fields

    HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    PRP = empymod.loop(Hsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)

    # Store in hypercube

    #Zcube[is1, is2, is3, it1, it2, 0:3] = HCP
    #Zcube[is1, is2, is3, it1, it2, 3:6] = VCP
    #Zcube[is1, is2, is3, it1, it2, 6:9] = PRP

    Z = np.hstack((HCP, VCP, PRP))
    return Z
            




In [26]:
startTime = time.time()

Results = Parallel(n_jobs=-1,verbose=10)(delayed(forward_parallel)(i, j, k, m, n) for i in range(nsl) 
                                         for j in range(nsl) for k in range(nsl) for m in range(nsl) for n in range (nsl))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   

Execution time in seconds: 56.2300808429718


[Parallel(n_jobs=-1)]: Done 3125 out of 3125 | elapsed:   56.1s finished


In [27]:
np.save('Results_nsl' + str(nsl), Results)